In [4]:
import sys
import pandas as pd
import datetime
import numpy as np
import time as _time
import requests
import pandas_datareader as pdr
from pandas_datareader import data as wb
from collections import defaultdict
# from yahoo_finance_api import YahooFinance as yf

class YahooFinance:
    def __init__(self, ticker, result_range='1mo', start=None, end=None, interval='15m', dropna=True):
        """
        Return the stock data of the specified range and interval
        Note - Either Use result_range parameter or use start and end
        Note - Intraday data cannot extend last 60 days
        :param ticker:  Trading Symbol of the stock should correspond with yahoo website
        :param result_range: Valid Ranges "1d","5d","1mo","3mo","6mo","1y","2y","5y","10y","ytd","max"
        :param start: Starting Date
        :param end: End Date
        :param interval:Valid Intervals - Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
        :return:
        """
        if result_range is None:
            start = int(_time.mktime(_time.strptime(start, '%d-%m-%Y')))
            end = int(_time.mktime(_time.strptime(end, '%d-%m-%Y')))
            # defining a params dict for the parameters to be sent to the API
            params = {'period1': start, 'period2': end, 'interval': interval}

        else:
            params = {'range': result_range, 'interval': interval}

        # sending get request and saving the response as response object
        url = "https://query1.finance.yahoo.com/v8/finance/chart/{}".format(ticker)
        r = requests.get(url=url, params=params)
        data = r.json()
        # Getting data from json
        error = data['chart']['error']
        if error:
            raise ValueError(error['description'])
        self._result = self._parsing_json(data)
        if dropna:
            self._result.dropna(inplace=True)

    @property
    def result(self):
        return self._result

    def _parsing_json(self, data):
        timestamps = data['chart']['result'][0]['timestamp']
        # Formatting date from epoch to local time
        timestamps = [_time.strftime('%a, %d %b %Y %H:%M:%S', _time.localtime(x)) for x in timestamps]
        volumes = data['chart']['result'][0]['indicators']['quote'][0]['volume']
        opens = data['chart']['result'][0]['indicators']['quote'][0]['open']
        opens = self._round_of_list(opens)
        closes = data['chart']['result'][0]['indicators']['quote'][0]['close']
        closes = self._round_of_list(closes)
        lows = data['chart']['result'][0]['indicators']['quote'][0]['low']
        lows = self._round_of_list(lows)
        highs = data['chart']['result'][0]['indicators']['quote'][0]['high']
        highs = self._round_of_list(highs)
        df_dict = {'Open': opens, 'High': highs, 'Low': lows, 'Close': closes, 'Volume': volumes}
        df = pd.DataFrame(df_dict, index=timestamps)
        df.index = pd.to_datetime(df.index)
        return df

    def _round_of_list(self, xlist):
        temp_list = []
        for x in xlist:
            if isinstance(x, float):
                temp_list.append(round(x, 2))
            else:
                temp_list.append(pd.np.nan)
        return temp_list

    def to_csv(self, file_name):
        self.result.to_csv(file_name)

symbol = '^NSEI'
data_period = '5d'
fin_prod_data = YahooFinance(symbol,result_range=data_period,interval='15m').result
# fin_prod_data = yf('reliance.ns',result_range='1mo',interval='15m').result
# fin_prod_data[('High')] = fin_prod_data[('High')] * height_precision
# fin_prod_data[('Low')] = fin_prod_data[('Low')] * height_precision
# fin_prod_data[('Close')] = fin_prod_data[('Close')] * height_precision
fin_prod_data = fin_prod_data.round({'Close':0, 'High': 0, 'Low': 0, 'Open': 0, })
# time_groups = fin_prod_data.groupby(pd.Grouper(freq='15min'))['Close'].mean()
# time_groups = time_groups.dropna()
distinct_dates = fin_prod_data.index.map(lambda t: t.date()).unique()

current_time_group_index = 0

from collections import defaultdict
mp = defaultdict(str)
char_mark = 64
# build dictionary with all needed prices
tot_min_price=min(np.array(fin_prod_data['Low']))
tot_max_price=max(np.array(fin_prod_data['High']))
yaxis_height_gap=int(round(((tot_max_price - tot_min_price) / 40)))
for price in range(int(tot_min_price), int(tot_max_price) + yaxis_height_gap, yaxis_height_gap):
    mp[price]+=('\t')
# add max price as it will be ignored in for range loop above
# mp[tot_max_price] = '\t' + str(time_groups.index[current_time_group_index])[5:7] + '/' + str(time_groups.index[current_time_group_index])[3:4]
mp[tot_max_price + yaxis_height_gap] = '\t' + str(distinct_dates[current_time_group_index])
x = 0
first_tpo = 1
while x < len(fin_prod_data):

    if fin_prod_data.index[x].date() > distinct_dates[current_time_group_index]:
        close_price = fin_prod_data['Close'][x-1]
        closeremainder = (close_price - tot_min_price) % yaxis_height_gap
        if closeremainder > (yaxis_height_gap / 2):
            close_price = close_price + (yaxis_height_gap - closeremainder)
        else:
            close_price = close_price - closeremainder
        mp[close_price]+='#'
        # new time period
        char_mark = 64
        first_tpo = 1
        # buffer and tab all entries
        buffer_max = max([len(v) for k,v in mp.iteritems()])
        # if current_time_group_index != len(time_groups) -1:
        current_time_group_index += 1
        for k,v in mp.iteritems():
            mp[k] += (chr(32) * (buffer_max - len(mp[k]))) + '\t'
        # mp[tot_max_price] += str(time_groups.index[current_time_group_index])[5:7] + '/' + str(time_groups.index[current_time_group_index])[3:4]
        mp[tot_max_price + yaxis_height_gap] += str(distinct_dates[current_time_group_index])


    char_mark += 1
    if first_tpo == 0 and x < len(fin_prod_data) - 1:
        min_price=min(fin_prod_data['Low'][x], fin_prod_data['Low'][x + 1])
        max_price=max(fin_prod_data['High'][x], fin_prod_data['High'][x + 1])
        minremainder =  (min_price - tot_min_price) % yaxis_height_gap
        maxremainder =  (tot_max_price - max_price) % yaxis_height_gap
        x+=2
    else:
        open_price = fin_prod_data['Open'][x]
        openremainder = (open_price - tot_min_price) % yaxis_height_gap
        min_price = fin_prod_data['Low'][x]
        max_price = fin_prod_data['High'][x]
        minremainder = (min_price - tot_min_price) % yaxis_height_gap
        maxremainder = (max_price - tot_min_price) % yaxis_height_gap
        # first_tpo = 0
        x+=1

    if minremainder > (yaxis_height_gap / 2):
        min_price = min_price + (yaxis_height_gap - minremainder)
    else:
        min_price = min_price - minremainder

    if maxremainder > (yaxis_height_gap / 2):
        max_price = max_price + (yaxis_height_gap - maxremainder)
    else:
        max_price = max_price - maxremainder

    if openremainder > (yaxis_height_gap / 2):
        open_price = open_price + (yaxis_height_gap - openremainder)
    else:
        open_price = open_price - openremainder



    for price in range(int(min_price), int(max_price) + yaxis_height_gap, yaxis_height_gap):
        if first_tpo == 1 and price == open_price:
            mp[price] += 'X'
            first_tpo = 0
        else:
            mp[price]+=(chr(char_mark))

close_price = fin_prod_data['Close'][x - 1]
closeremainder = (close_price - tot_min_price) % yaxis_height_gap
if closeremainder > (yaxis_height_gap / 2):
    close_price = close_price + (yaxis_height_gap - closeremainder)
else:
    close_price = close_price - closeremainder
mp[close_price] += '#'
sorted_keys = sorted(mp.keys(), reverse=True)
for y in sorted_keys:
    # buffer each list
    print(str("{0:.2f}".format((y * 1.0))) + ': \t' + ''.join(mp[y]))




10851.00: 		2018-12-10	2018-12-11	2018-12-12	2018-12-13  	2018-12-14
10842.00: 		          	          	          	J           	
10829.00: 		          	          	          	IJ          	B
10816.00: 		          	          	          	ABCDHIJKM   	BCDHIM
10803.00: 		          	          	          	XBCDEFGHIKLM	BCDEFGHIJKLM#
10790.00: 		          	          	          	ABDEFGHKLM# 	ABCDEFGHIJKLM
10777.00: 		          	          	          	BEFGKLM     	ABDEFGJK
10764.00: 		          	          	M         	KLM         	XBEF
10751.00: 		          	          	M         	L           	A
10738.00: 		          	          	LM#       	            	
10725.00: 		          	          	LM        	            	
10712.00: 		          	          	L         	            	
10699.00: 		          	          	KL        	            	
10686.00: 		          	          	DJKL      	            	
10673.00: 		          	          	CDEFGHJK  	            	
10660.00: 		          	          	CDEFGHJ   	            	
